In [ ]:
with open('ratings.dat') as f:
    lines = f.readlines()
user_history = {}
for line in lines:
    data = line.strip().split("::")
    user_id = data[0]
    movie_id = data[1]
    time_stamp = int(data[3])
    if user_id in user_history:
        user_history[user_id].append((time_stamp, movie_id))
    else:
        user_history[user_id] = [(time_stamp, movie_id)]

for s in user_history.keys():
    l = sorted(user_history[s])
    user_history[s] = l


In [ ]:
import time
import csv
import pickle

import operator
#one hour session or one day
session_threshold = 86400 #3600

In [ ]:
def extract_session(user_his, threshold):
    result = []
    session = []
    start_time = user_his[0][0]
    for i in range(len(user_his)):
        time = user_his[i][0]
        if time - start_time <= threshold:
            session.append(user_his[i])
        else:
            result.append(session)
            session = [user_his[i]]
            start_time = user_his[i][0]
    if session not in result:
        result.append(session)
    return result

In [ ]:
session = []
for i in user_history.keys():
    session += extract_session(user_history[i], session_threshold)

In [ ]:
filtered_session = []
for s in session:
    if 2 <= len(s) and len(s) <= 50:
        filtered_session.append(s)

In [ ]:
# Count number of times each item appears
iid_counts = {}
for seq in filtered_session:
    for iid in seq:
        if iid[1] in iid_counts:
            iid_counts[iid[1]] += 1
        else:
            iid_counts[iid[1]] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))


In [ ]:
final_session = []
for curseq in filtered_session:
    filseq = list(filter(lambda i: iid_counts[i[1]] >= 10, curseq))
    if len(filseq) < 2:
        continue
    else:
        final_session.append(filseq)

In [ ]:
maxdate = final_session[0][0][0]

for s in final_session:
    for item in s:
        if maxdate < item[0]:
            maxdate = item[0]

In [ ]:
#last two years as test
split_date = maxdate - 86400 * 730

In [ ]:
train_sess = filter(lambda x: x[-1][0] < split_date, final_session)
test_sess = filter(lambda x: x[-1][0] > split_date, final_session)
train_sess = sorted(train_sess, key=operator.itemgetter(1))
test_sess = sorted(test_sess, key=operator.itemgetter(1))

In [ ]:
# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
item_ctr = 1
train_seqs = []
# Convert training sessions to sequences and renumber items to start from 1
for seq in train_sess:
    outseq = []
    for i in seq:
        if i[1] in item_dict:
            outseq += [item_dict[i[1]]]
        else:
            outseq += [item_ctr]
            item_dict[i[1]] = item_ctr
            item_ctr += 1
    if len(outseq) < 2:  # Doesn't occur
        continue
    train_seqs += [outseq]

In [ ]:
test_seqs = []
# Convert test sessions to sequences, ignoring items that do not appear in training set
for seq in test_sess:
    outseq = []
    for i in seq:
        if i[1] in item_dict:
            outseq += [item_dict[i[1]]]
    if len(outseq) < 2:
        continue
    test_seqs += [outseq]

In [ ]:
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]

    return out_seqs, labs


tr_seqs, tr_labs = process_seqs(train_seqs)
te_seqs, te_labs = process_seqs(test_seqs)

train = (tr_seqs, tr_labs)
test = (te_seqs, te_labs)

In [ ]:
f1 = open('./movie_train.pkl', 'wb')
pickle.dump(train, f1)
f1.close()
f2 = open('./movie_test.pkl', 'wb')
pickle.dump(test, f2)
f2.close()

print('Done.') 

Done.


In [ ]:
genre_dic = {}
genre_index = 1

movie_strid_to_genres = {}
with open('movies.dat') as f:
    lines = f.readlines()
for line in lines:
    data = line.strip().split("::")
    movie_id = data[0]
    genres_string = data[2].split("|")
    genres = []
    for s in genres_string:
        if s not in genre_dic:
            genre_dic[s] = genre_index
            genre_index += 1
        genres.append(genre_dic[s])
    genres = np.array(genres)
    movie_strid_to_genres[movie_id] = genres

In [ ]:
movie_intid_to_genres = {}
for string in item_dict.keys():
    movie_intid_to_genres[item_dict[string]] = movie_strid_to_genres[string]

In [ ]:
f2 = open('./movie_side.pkl', 'wb')
pickle.dump(movie_intid_to_genres, f2)
f2.close()
movie_intid_to_genres
